# SKLearn 07 - *Regression* dengan KNN (*K-Nearest Neighbours*)

- KNN merupakan suatu model machine learning yang dapat digunakan untuk **melakukan prediksi berdasarkan kedekatan karakteristik dengan sejumlah tetangga terdekatnya**.
- Prediksi yang dilakukan dapat diterapkan baik pada ***classification*** maupun ***regression tasks***.

## *Sample Dataset*
- Dataset akan digenerate sebagai pandas data frame, sehingga kita perlu melakukan import **pandas**
> `import pandas as pd`
- Kali ini kita akan membuat dataset yang dibentuk menggunakan dictionary yang terdiri atas keys (**tinggi**, **jk**, dan **berat**) dan valuesnya. Dataset tersebut akan ditampung oleh variabel **sensus**. Kolom tinggi dan jk akan menjadi features, sedangkan berat akan menjadi target.
> `sensus = {
    'tinggi': [158, 170, 183, 191, 155, 163, 180, 158, 178], 
    'jk': ['pria', 'pria', 'pria', 'pria', 'wanita', 'wanita', 'wanita', 'wanita','wanita'],
    'berat': [64, 86, 84, 80, 49, 59, 67, 54, 67]
}`
- Membuat pandas data frame dari **sensus** yang hasilnya akan ditampung oleh **sensus_df**.
> `sensus_df = pd.DataFrame(sensus)`
- Menampilkan dataset yang ditampung oleh **sensus_df**
> `sensus_df`

In [1]:
import pandas as pd

sensus = {'tinggi': [158, 170, 183, 191, 155, 163, 180, 158, 170], 
          'jk': ['pria', 'pria', 'pria', 'pria', 'wanita', 'wanita', 'wanita', 'wanita', 'wanita'],
          'berat': [64, 86, 84, 80, 49, 59, 67, 54, 67]}

sensus_df = pd.DataFrame(sensus)
sensus_df

,tinggi,jk,berat
0,158,pria,64
1,170,pria,86
2,183,pria,84
3,191,pria,80
4,155,wanita,49
5,163,wanita,59
6,180,wanita,67
7,158,wanita,54
8,170,wanita,67


## *Regression* dengan KNN
### Features & Target
Untuk memodelkan hubungan antara tinggi badan dan jenis kelamin dengan berat badan, kita perlu untuk mengelompokkan features dan targetnya menggunakan numpy array.
- Mengimport **numpy** sebagai np.
> `import numpy as np`
- Mengkonversikan pandas data frame **sensus_df** menjadi **numpy array**. Kita akan membuat dua buah numpy array yang pertama bertindak sebagai features (**tinggi** dan **jk**) dan yang kedua bertindak sebagai target (**berat**). 
> `X_train = np.array(sensus_df[['tinggi', 'jk']])`

    > `y_train = np.array(sensus_df['berat'])`
- Menampilkan sekumpulan nilai features `X_train` dan target `y_train`.
> `print(f'X_train:\n{X_train}\n')`

    > `print(f'y_train: {y_train}')`

In [2]:
import numpy as np

X_train = np.array(sensus_df[['tinggi', 'jk']])
y_train = np.array(sensus_df['berat'])

print(f'X_train:\n{X_train}\n')
print(f'y_train: {y_train}')

X_train:
[[158 'pria']
 [170 'pria']
 [183 'pria']
 [191 'pria']
 [155 'wanita']
 [163 'wanita']
 [180 'wanita']
 [158 'wanita']
 [170 'wanita']]

y_train: [64 86 84 80 49 59 67 54 67]


### Preprocess Dataset: Konversi Label menjadi Numerik Biner
- Melakukan transpose pada X_train sehingga kolom tinggi menjadi baris dengan index 0 dan jk menjadi baris dengan index 1 pada dataset.
> `X_train_transposed = np.transpose(X_train)`

In [3]:
X_train_transposed = np.transpose(X_train)

print(f'X_train:\n{X_train}\n')
print(f'X_train_transposed:\n{X_train_transposed}')

X_train:
[[158 'pria']
 [170 'pria']
 [183 'pria']
 [191 'pria']
 [155 'wanita']
 [163 'wanita']
 [180 'wanita']
 [158 'wanita']
 [170 'wanita']]

X_train_transposed:
[[158 170 183 191 155 163 180 158 170]
 ['pria' 'pria' 'pria' 'pria' 'wanita' 'wanita' 'wanita' 'wanita'
  'wanita']]


Mentransform nilai target `y_train` dari String ke nilai numerik biner dimana `pria` menjadi `0` dan `wanita` menjadi `1` agar dapat dihitung jaraknya. 
- Mengimport `LabelBinarizer` dari `sklearn.preprocessing`
- Membuat variabel `lb` yang akan menampung object `LabelBinarizer()`
- Memanggil `lb.fit_transform(X_train_transposed[1])` untuk melakukan transform label X_train_transposed[1] atau baris dengan index ke-1 yang berisikan data jk yang hasil transformnya akan ditampung oleh `jk_binarised`.
- Menampilkan X_train_transposed[1] dan jk_binarised.

In [4]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
jk_binarised = lb.fit_transform(X_train_transposed[1])

print(f'jk: {X_train_transposed[1]}\n')
print(f'jk_binarised:\n{jk_binarised}')

jk: ['pria' 'pria' 'pria' 'pria' 'wanita' 'wanita' 'wanita' 'wanita' 'wanita']

jk_binarised:
[[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]


- Mengkonversi jk_binarised yang terdiri dari array 2 dimensi menjadi array 1 dimensi dengan memanggil method `flatten()`.

In [5]:
jk_binarised = jk_binarised.flatten()
jk_binarised

array([0, 0, 0, 0, 1, 1, 1, 1, 1])

- Melakukan transpose kembali pada X_train_transposed sehingga tinggi dan jenis kelamin kembali menjadi kolom dimana label pada jenis kelamin laki-laki dan wanita telah dikonversi menjadi numerik biner 0 dan 1
> `X_train = X_train_transposed.transpose()`

In [6]:
X_train_transposed[1] = jk_binarised
X_train = X_train_transposed.transpose()

print(f'X_train_transposed:\n{X_train_transposed}\n')
print(f'X_train:\n{X_train}')

X_train_transposed:
[[158 170 183 191 155 163 180 158 170]
 [0 0 0 0 1 1 1 1 1]]

X_train:
[[158 0]
 [170 0]
 [183 0]
 [191 0]
 [155 1]
 [163 1]
 [180 1]
 [158 1]
 [170 1]]


### *Training KNN Regression Model*
- Mengimport ***KNeighborsClassifier*** dari ***sklearn.neighbors**
> `from sklearn.linear_model import LinearRegression`
- Membuat object KNN Classification Model dimana nilai dari `n_neigbors`nya adalah 3.
> `K = 3`
    
    > `model = KNeighborsClassifier(n_neighbors=K)`
- Untuk melakukan training model, kita dapat memanggil method **fit()** yang disertai 2 parameter.
    - Parameter pertama berasosiasi dengan sekumpulan nilai features yaitu **X_train**.
    - Parameter kedua berasosiasi dengan sekumpulan nilai target yaitu **y_train**.
> `model.fit(X_train, y_train)`

In [7]:
from sklearn.neighbors import KNeighborsRegressor

K = 3
model = KNeighborsRegressor(n_neighbors=K)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

### Prediksi Berat Badan
Kita akan melakukan prediksi berat badan untuk seseorang yang memiliki tinggi badan 155 cm dan berjenis kelamin wanita (1).
> `tinggi_badan = 155`

> `jenis_kelamin = 1`

- Membuat variable **X_new** yang akan menampung numpy array yang terdiri dari 2 nilai yaitu tinggi_badan dan jenis_kelamin yang kemudian dilakukan reshape agar menghasilkan numpy array 2 dimensi.
> `X_new = np.array([tinggi_badan, jenis_kelamin]).reshape(1, -1)`
- Menampilkan X_new
> `X_new`

In [8]:
tinggi_badan = 155
jenis_kelamin = 1
X_new = np.array([tinggi_badan, jenis_kelamin]).reshape(1, -1)
X_new

array([[155,   1]])

- Memanggil `model.predict()` yang akan melakukan prediksi nilai target dari features X_new yang ditentukan sebelumnya. Hasil prediksi ditampung oleh variabel `y_new`.

In [9]:
y_new = model.predict(X_new)
y_new

array([55.66666667])

## Evaluasi KNN Regression Model
### Testing Set
- Membuat numpy array dari sekumpulan nilai features (tinggi badan dan jenis kelamin) untuk testing dataset. Numpy array tersebut akan ditampung oleh variabel `X_test`.
> `X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])`
- Membuat numpy array dari sekumpulan nilai target (berat badan) untuk testing dataset. Numpy array tersebut akan ditampung oleh variabel `y_test`.
> `y_test = np.array([65, 96, 52, 67])`

In [10]:
X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])
y_test = np.array([65, 96, 52, 67])

print(f'X_test:\n{X_test}\n')
print(f'y_test: {y_test}')

X_test:
[[168   0]
 [180   0]
 [160   1]
 [169   1]]

y_test: [65 96 52 67]


### Prediksi terhadap Testing Set
- Memanggil `model.predict(X_test)` untuk memprediksi nilai target dari features yang ditampung variabel X_test. Hasil prediksi tersebut akan ditampung oleh variabel `y_pred`
> `y_pred = model.predict(X_test)`

In [11]:
y_pred = model.predict(X_test)
y_pred

array([70.66666667, 79.        , 59.        , 70.66666667])

### *Coefficient of Determination* atau *R-squared* ($R^2$)
Untuk melakukan evaluasi terhadap KNN Regression Model, kita dapat memanfaatkan R-squared. Nilai R-Squared semakin mendekati 1 berarti semakin baik model yang dihasilkan, sedangkan jika semakin mendekati 0 atau bahkan negatif berarti semakin buruk model yang dihasilkan dalam melakukan prediksi.
- Mengimport **r2_score** dari **sklearn.metrics**
> `from sklearn.metrics import r2_score`
- Membandingkan sekumpulan nilai target yang ditampung oleh **y_test** dengan hasil prediksi yang ditampung **y_pred**. Proses evaluasi ini menggunakan method `r2_score()` yang akan menghasilkan nilai yang merepresentasikan kesesuaian dari kedua dataset tersebut. Semakin bagus dan akurat model yang dihasilkan, nilainya akan mendekati angka 1.
> `r_squared = r2_score(y_test, y_pred)`

In [12]:
from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)

print(f'R-squared: {r_squared}')

R-squared: 0.6290565226735438


### *Mean Absolute Error* (MAE) atau *Mean Absolute Deviation* (MAD)
- MAE merupakan rata-rata nilai absolute value dari error yang terjadi pada hasil prediksi

$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$

- $y_i$ pada kasus ini adalah y_test dan $\hat{y}_i$ adalah y_pred.
- Jika nilai y_pred lebih kecil dari y_test, nilai errornya akan positif. Sedangkan jika nilai y_pred lebih besar dari y_test, nilai errornya akan negatif. Untuk mengatasi hal tersebut proses pengurangan y_test oleh y_pred dikenakan operasi nilai mutlak, sehingga nilai errornya yang dihasilkannya selalu positif. 
- Semakin kecil MAE merepresentasikan semakin baik model yang dihasilkan dalam melakukan prediksi.
- Semakin besar MAE merepresentasikan semakin buruk model yang dihasilkan dalam melakukan prediksi.

In [13]:
from sklearn.metrics import mean_absolute_error

MAE = mean_absolute_error(y_test, y_pred)

print(f'MAE: {MAE}')

MAE: 8.333333333333336


### *Mean Squared Error* (MSE) atau *Mean Squared Deviation* (MSD)
- MSE merupakan rata-rata dari kuadrat nilai error yang terjadi pada hasil prediksi

$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$

- $y_i$ pada kasus ini adalah y_test dan $\hat{y}_i$ adalah y_pred.
- Jika nilai y_pred lebih kecil dari y_test, nilai errornya akan positif. Sedangkan jika nilai y_pred lebih besar dari y_test, nilai errornya akan negatif. Untuk mengatasi hal tersebut proses pengurangan y_test oleh y_pred dikenakan operasi kuadrat, sehingga nilai errornya yang dihasilkannya selalu positif. 
- Semakin kecil MSE merepresentasikan semakin baik model yang dihasilkan dalam melakukan prediksi.
- Semakin besar MSE merepresentasikan semakin buruk model yang dihasilkan dalam melakukan prediksi.

In [14]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test, y_pred)

print(f'MSE: {MSE}')

MSE: 95.8888888888889


### Permasalahan *Scaling* pada *Features*

Permasalahan Scaling pada features ini bisa terjadi karena perbedaan satuan yang digunakan, misalnya milimeter dan meter. Dengan perbedaan satuan tersebut bisa memberikan hasil yang tidak konsisten mengenai data point mana di dalam dataset yang memiliki jarak yang lebih dekat dengan data point yang kita cari.

**Tinggi dalam milimeter**
- Berikut ini kita akan menggunakan satuan milimeter untuk nilai dari tinggi badan. 
- Nilai euclidean yang dihasilkan menyatakan bahwa jarak antara X_new[0] dan X_train[0][0] lebih besar dibandingkan dengan jarak antara X_new[0] dan X_train[1][0]

In [15]:
from scipy.spatial.distance import euclidean

X_train = np.array([[1700, 0], [1600, 1]])
X_new = np.array([[1640, 0]])

[euclidean(X_new[0], d) for d in X_train]

[60.0, 40.01249804748511]

**Tinggi dalam meter**
- Berikut ini kita akan menggunakan satuan meter untuk nilai dari tinggi badan. 
- Nilai euclidean yang dihasilkan menyatakan bahwa jarak antara X_new[0] dan X_train[0][0] lebih kecil dibandingkan dengan jarak antara X_new[0] dan X_train[1][0]

In [16]:
X_train = np.array([[1.7, 0], [1.6, 1]])
X_new = np.array([[1.64, 0]])

[euclidean(X_new[0], d) for d in X_train]

[0.06000000000000005, 1.0007996802557442]

### Menerapkan *Standard Scaler* (*Standard Score* atau *Z-Score*)

Untuk mengatasi permasalahan sebelumnya, kita dapat menerapkan *Standard Scaler* (*Standard Score* atau *Z-Score*)
- Standardize features by removing the mean and scaling to unit variance.

$z = \frac{x - \bar{x}}{S}$

In [17]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

**Tinggi dalam milimeter**
- Berikut ini kita akan menggunakan satuan milimeter untuk nilai dari tinggi badan. 
- Nilai euclidean yang dihasilkan menyatakan bahwa jarak antara X_new[0] dan X_train[0][0] lebih kecil dibandingkan dengan jarak antara X_new[0] dan X_train[1][0]

In [18]:
X_train = np.array([[1700, 0], [1600, 1]])
X_train_scaled = ss.fit_transform(X_train)
print(f'X_train_scaled:\n{X_train_scaled}\n')

X_new = np.array([[1640, 0]])
X_new_scaled = ss.transform(X_new)
print(f'X_new_scaled: {X_new_scaled}\n')

jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]
print(f'jarak: {jarak}')

X_train_scaled:
[[ 1. -1.]
 [-1.  1.]]

X_new_scaled: [[-0.2 -1. ]]

jarak: [1.2, 2.1540659228538015]


**Tinggi dalam meter**
- Berikut ini kita akan menggunakan satuan milimeter untuk nilai dari tinggi badan. 
- Nilai euclidean yang dihasilkan menyatakan bahwa jarak antara X_new[0] dan X_train[0][0] lebih kecil dibandingkan dengan jarak antara X_new[0] dan X_train[1][0]
- Jadi, baik menggunakan satuan milimeter maupun menggunakan satuan milimeter, nilai euclidean yang dihasilkan sama-sama menyatakan bahwa arak antara X_new[0] dan X_train[0][0] lebih kecil dibandingkan dengan jarak antara X_new[0] dan X_train[1][0].

In [19]:
X_train = np.array([[1.7, 0], [1.6, 1]])
X_train_scaled = ss.fit_transform(X_train)
print(f'X_train_scaled:\n{X_train_scaled}\n')

X_new = np.array([[1.64, 0]])
X_new_scaled = ss.transform(X_new)
print(f'X_new_scaled: {X_new_scaled}\n')

jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]
print(f'jarak: {jarak}')

X_train_scaled:
[[ 1. -1.]
 [-1.  1.]]

X_new_scaled: [[-0.2 -1. ]]

jarak: [1.2000000000000026, 2.1540659228538006]


## Menerapkan *Features Scaling* pada KNN
### Training Dataset

In [20]:
X_train = np.array([[158, 0], [170, 0], [183, 0], [191, 0], [155, 1], [163, 1],
                    [180, 1], [158, 1], [170, 1]])

y_train = np.array([64, 86, 84, 80, 49, 59, 67, 54, 67])

### Testing Dataset

In [21]:
X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])
y_test = np.array([65, 96, 52, 67])

### *Features Scaling* (*Standard Scaler*)

In [22]:
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

print(f'X_train_scaled:\n{X_train_scaled}\n')
print(f'X_test_scaled:\n{X_test_scaled}\n')

X_train_scaled:
[[-0.9908706  -1.11803399]
 [ 0.01869567 -1.11803399]
 [ 1.11239246 -1.11803399]
 [ 1.78543664 -1.11803399]
 [-1.24326216  0.89442719]
 [-0.57021798  0.89442719]
 [ 0.86000089  0.89442719]
 [-0.9908706   0.89442719]
 [ 0.01869567  0.89442719]]

X_test_scaled:
[[-0.14956537 -1.11803399]
 [ 0.86000089 -1.11803399]
 [-0.82260955  0.89442719]
 [-0.06543485  0.89442719]]



### *Training* & Evaluasi Model

In [23]:
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)

print(f'MAE: {MAE}')
print(f'MSE: {MSE}')

MAE: 7.583333333333336
MSE: 85.13888888888893
